In [ ]:
# Importowanie potrzebnych bibliotek
import openai  
import time  
import os  
from PyPDF2 import PdfReader 
from pinecone import Pinecone, ServerlessSpec  

from dotenv import load_dotenv
load_dotenv()  

In [ ]:
# Konfiguracja kluczy API  
pinecone_api_key = ''
openai.api_key = ''  

In [ ]:
# Inicjalizacja bazy wektorowej Pinecone  
pc = Pinecone(api_key=pinecone_api_key)  
index_name = 'chatbot'  

In [ ]:
# Sprawdzenie, czy indeks w bazie wektorowej istnieje, jeśli nie - utworzenie go  
if index_name not in pc.list_indexes().names():  
    pc.create_index(  
        name=index_name,  
        dimension=1536,  
        metric='cosine',  
        spec=ServerlessSpec(  
            cloud='aws',  
            region='us-east-1'  
        )  
    )  
    # Czekanie, aż indeks zostanie zainicjalizowany  
    while not pc.describe_index(index_name).status['ready']:  
        time.sleep(1)  

In [ ]:
# Połączenie z indeksem  
index = pc.Index(index_name)  

In [ ]:
# Funkcja do czytania tekstu z pliku PDF 
def read_pdf(file_path):  
    with open(file_path, 'rb') as file:  
        reader = PdfReader(file)  
        text = ''  
        for page_num in range(len(reader.pages)):  
            page = reader.pages[page_num]  
            text += page.extract_text()  
    return text  

# Funkcja do chunkowania tekstu  
def chunk_text(text, chunk_size=500, overlap=50):  
    chunks = []  
    start = 0  
    while start < len(text):  
        end = start + chunk_size  
        chunk = text[start:end]  
        chunks.append(chunk)  
        start += chunk_size - overlap  
    return chunks 

# Funkcja do przetwarzania pliku PDF, chunkowania go i wrzucania do bazy Pinecone  
def process_pdf_and_upsert(file_path, index):  
    text = read_pdf(file_path)  
    chunks = chunk_text(text)  
    upsert_embeddings(chunks, index)  

# Funkcja do embedowania tekstów  
def get_embedding(text, model="text-embedding-3-small"):  
    text = text.replace("\n", " ")  
    response = openai.Embedding.create(input=[text], model=model)  
    return response['data'][0]['embedding']  
  
# Funkcja do wrzucania embeddingów do bazy Pinecone  
def upsert_embeddings(texts, index):  
    embeddings = [get_embedding(text) for text in texts]  
    ids = [str(i) for i in range(len(texts))]  
    meta = [{'text': text} for text in texts]  
    to_upsert = list(zip(ids, embeddings, meta))  
    index.upsert(vectors=to_upsert)  
  
# Funkcja do wykonywania zapytań do bazy Pinecone i zwracania najlepszego chunku informacji  
def query_embedding(query, index, model="text-embedding-3-small"):  
    query_embedding = get_embedding(query, model=model)  
    result = index.query(vector=query_embedding, top_k=1, include_metadata=True)  
    return result['matches'][0]['metadata']['text']   

# Funkcja do generowania odpowiedzi na pytanie na podstawie najlepszego chunku  
def generate_answer(query, index):  
    best_match_text = query_embedding(query, index)  
    prompt = f"Based on the following information, answer the question: '{query}'\n\nInformation: {best_match_text}"  
      
    response = openai.ChatCompletion.create(  
        model="gpt-4o",  
        messages=[  
            {"role": "system", "content": "Jesteś pomocnym asystentem."},  
            {"role": "user", "content": prompt}]
        )  
      
    return response.choices[0].message.content

In [ ]:
# Ścieżka do pliku PDF  
pdf_file = os.path.join('Przewodnik_po_historii_Polski.pdf')  

# Przetwarzanie pliku PDF, chunkowanie i wrzucanie do bazy Pinecone  
process_pdf_and_upsert(pdf_file, index) 

In [ ]:
# Zadanie pytania i generowanie odpowiedzi  
query = "W którym roku opierając się na jałtańskich decyzjach Wielkiej Trójki, pod kontrolą Moskwy utworzono rząd, który miał zorganizować w okrojonym kraju wybory parlamentarne?"  
answer = generate_answer(query, index)  
print(f"Answer: {answer}")  